In [1]:
using IntervalArithmetic
using LaTeXStrings
using Plots

In [2]:
include("./method3/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")

print()

In [3]:
function laTeXMatrix(
        pattern,
        size::Tuple{Integer, Integer})::String
    m, n = size

    mat = "\\begin{bmatrix}"
    for i = 1:m, j = 1:n
        mat *= pattern(i, j)
        if j == n
            if i != m
                mat *= " \\\\ "
            end
        else
            mat *= " & "
        end
    end
    mat *= "\\end{bmatrix}"

    return mat
end

laTeXMatrix (generic function with 1 method)

In [6]:
function method3(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    _, n = size(A₁)

    ε = solveFeasibilityCheckLP_m3.([A₁, A₂])
    for k = eachindex(ε)
        println("\\varepsilon_$k^* = $(string(round(ε[k], digits=3)))"); println()
    end

    approximationResults = solveApproximationLP_m3.([A₁, A₂])
    for k = eachindex(approximationResults)
        resultₛₜᵣ = approximationLPResultLaTeXString_m3(approximationResults[k])

        if resultₛₜᵣ.Wₖ⁻ == ""
            println("W_$k^- = \\emptyset"); println()
            println("W_$k^+ = $(resultₛₜᵣ.Wₖ⁺)"); println()
        else
            println("W_$k^- = $(resultₛₜᵣ.Wₖ⁻)"); println()
            println("W_$k^+ = $(resultₛₜᵣ.Wₖ⁺)"); println()
        end
    end

    tBoundaries = calculateTBoundaries_m3.(approximationResults)
    for k = eachindex(tBoundaries)
        boundariesₛₜᵣ = tBoundariesLaTeXString_m3(tBoundaries[k])

        tₖᴸ⁻ = boundariesₛₜᵣ.tₖᴸ⁻
        tₖᵁ⁻ = boundariesₛₜᵣ.tₖᵁ⁻
        tₖᴸ⁺ = boundariesₛₜᵣ.tₖᴸ⁺
        tₖᵁ⁺ = boundariesₛₜᵣ.tₖᵁ⁺
    
        if tₖᴸ⁻  == "" || tₖᵁ⁻ == ""
            println("""
                t_{$(k)}^{\\text{L}+} = $(tₖᴸ⁺), ~~
                t_{$(k)}^{\\text{U}+} = $(tₖᵁ⁺)
            """); println()
        else
            println("""
                t_{$(k)}^{\\text{L}-} = $(tₖᴸ⁻), ~~
                t_{$(k)}^{\\text{U}-} = $(tₖᵁ⁻), ~~
                t_{$(k)}^{\\text{L}+} = $(tₖᴸ⁺), ~~
                t_{$(k)}^{\\text{U}+} = $(tₖᵁ⁺)
            """); println()
        end
    end

    result = solveConcatLP_m3(approximationResults, tBoundaries)
    resultₛₜᵣ = concatLPResultLaTeXString_m3(result)
    m = length(result.t⁻)
    println("""
        $(laTeXMatrix((i,j) -> "t_$i^-", (m, 1))) = $(resultₛₜᵣ.t⁻), ~~
        $(laTeXMatrix((i,j) -> "t_$i^+", (m, 1))) = $(resultₛₜᵣ.t⁺)
        """); println()
    println("W = $(resultₛₜᵣ.W)"); println()
    println("""
        $(laTeXMatrix((i,j) -> "v_$i^{\\text{L}-}", (n, 1))) = $(resultₛₜᵣ.vᴸ⁻), ~~
        $(laTeXMatrix((i,j) -> "v_$i^{\\text{U}-}", (n, 1))) = $(resultₛₜᵣ.vᵁ⁻), ~~
        $(laTeXMatrix((i,j) -> "v_$i^{\\text{L}+}", (n, 1))) = $(resultₛₜᵣ.vᴸ⁺), ~~
        $(laTeXMatrix((i,j) -> "v_$i^{\\text{U}+}", (n, 1))) = $(resultₛₜᵣ.vᵁ⁺)
        """); println()
    println("""
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{L}", (n, 1))) = $(resultₛₜᵣ.εᴸ), ~~
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{U}", (n, 1))) = $(resultₛₜᵣ.εᵁ)
        """); println()

    # Â = generatePCM_m3(result)

    return
end

method3 (generic function with 1 method)

# example 1

In [9]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
A₂ = A₁

method3(A₁, A₂)

\varepsilon_1^* = 0.0

\varepsilon_2^* = 0.0

W_1^- = \begin{bmatrix} \left[ 0.314, 0.314 \right]  \\  \left[ 0.157, 0.157 \right]  \\  \left[ 0.294, 0.294 \right]  \\  \left[ 0.235, 0.235 \right] \end{bmatrix}

W_1^+ = \begin{bmatrix} \left[ 0.235, 0.314 \right]  \\  \left[ 0.157, 0.235 \right]  \\  \left[ 0.235, 0.353 \right]  \\  \left[ 0.235, 0.235 \right] \end{bmatrix}

W_2^- = \begin{bmatrix} \left[ 0.314, 0.314 \right]  \\  \left[ 0.157, 0.157 \right]  \\  \left[ 0.294, 0.294 \right]  \\  \left[ 0.235, 0.235 \right] \end{bmatrix}

W_2^+ = \begin{bmatrix} \left[ 0.235, 0.314 \right]  \\  \left[ 0.157, 0.235 \right]  \\  \left[ 0.235, 0.353 \right]  \\  \left[ 0.235, 0.235 \right] \end{bmatrix}

    t_{1}^{\text{L}-} = 1.0, ~~
    t_{1}^{\text{U}-} = 1.0, ~~
    t_{1}^{\text{L}+} = 0.981, ~~
    t_{1}^{\text{U}+} = 1.02


    t_{2}^{\text{L}-} = 1.0, ~~
    t_{2}^{\text{U}-} = 1.0, ~~
    t_{2}^{\text{L}+} = 0.981, ~~
    t_{2}^{\text{U}+} = 1.02


\begin{bmatrix}t_1^- \\ t_2^-\en

# example 2

In [10]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4 / 5)..(A₁[i,j].hi * 5 / 4)
end

method3(A₁, A₂)

\varepsilon_1^* = 0.0

\varepsilon_2^* = 0.0

W_1^- = \begin{bmatrix} \left[ 0.314, 0.314 \right]  \\  \left[ 0.157, 0.157 \right]  \\  \left[ 0.294, 0.294 \right]  \\  \left[ 0.235, 0.235 \right] \end{bmatrix}

W_1^+ = \begin{bmatrix} \left[ 0.235, 0.314 \right]  \\  \left[ 0.157, 0.235 \right]  \\  \left[ 0.235, 0.353 \right]  \\  \left[ 0.235, 0.235 \right] \end{bmatrix}

W_2^- = \begin{bmatrix} \left[ 0.343, 0.343 \right]  \\  \left[ 0.137, 0.137 \right]  \\  \left[ 0.313, 0.313 \right]  \\  \left[ 0.206, 0.206 \right] \end{bmatrix}

W_2^+ = \begin{bmatrix} \left[ 0.206, 0.343 \right]  \\  \left[ 0.137, 0.258 \right]  \\  \left[ 0.206, 0.386 \right]  \\  \left[ 0.206, 0.258 \right] \end{bmatrix}

    t_{1}^{\text{L}-} = 1.0, ~~
    t_{1}^{\text{U}-} = 1.0, ~~
    t_{1}^{\text{L}+} = 0.981, ~~
    t_{1}^{\text{U}+} = 1.02


    t_{2}^{\text{L}-} = 1.0, ~~
    t_{2}^{\text{U}-} = 1.0, ~~
    t_{2}^{\text{L}+} = 0.94, ~~
    t_{2}^{\text{U}+} = 1.069


\begin{bmatrix}t_1^- \\ t_2^-\en

<!-- ともに弱い整合性を持ち， $A_{1ij} \subseteq A_{2ij}$

```
A₁ = [
    1..1 2..5 2..4 2..3
    (1/5)..(1/2) 1..1 (1/5)..4 (1/5)..3
    (1/4)..(1/2) (1/4)..5 1..1 (1/4)..3
    (1/3)..(1/2) (1/3)..5 (1/3)..4 1..1
]
A₂ = fill(1..1, size(A₁))
_, n = size(A₁)
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 1/1.1)..(A₁[i,j].hi * 1.1)
end

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

display(isConsistentIntervalPCM(A₁))
display(isWeaklyConsistentIntervalPCM(A₁))
display(isConsistentIntervalPCM(A₂))
display(isWeaklyConsistentIntervalPCM(A₂))

plotAll(A₁, A₂)
``` -->

# example 3

In [11]:
A = [
    1 2 4 4
    1/2 1 2 2
    1/4 1/2 1 1
    1/4 1/2 1 1
]

A₁ = randamizedIntervalPCM(A, 1234, 3e-2)
A₂ = randamizedIntervalPCM(A, 134, 3e-2)

method3(A₁, A₂)

\varepsilon_1^* = 0.0

\varepsilon_2^* = 0.0

W_1^- = \begin{bmatrix} \left[ 0.501, 0.501 \right]  \\  \left[ 0.249, 0.249 \right]  \\  \left[ 0.126, 0.126 \right]  \\  \left[ 0.124, 0.124 \right] \end{bmatrix}

W_1^+ = \begin{bmatrix} \left[ 0.501, 0.501 \right]  \\  \left[ 0.246, 0.253 \right]  \\  \left[ 0.122, 0.127 \right]  \\  \left[ 0.124, 0.127 \right] \end{bmatrix}

W_2^- = \begin{bmatrix} \left[ 0.501, 0.501 \right]  \\  \left[ 0.249, 0.249 \right]  \\  \left[ 0.126, 0.126 \right]  \\  \left[ 0.124, 0.124 \right] \end{bmatrix}

W_2^+ = \begin{bmatrix} \left[ 0.501, 0.501 \right]  \\  \left[ 0.247, 0.251 \right]  \\  \left[ 0.124, 0.127 \right]  \\  \left[ 0.123, 0.127 \right] \end{bmatrix}

    t_{1}^{\text{L}-} = 1.0, ~~
    t_{1}^{\text{U}-} = 1.0, ~~
    t_{1}^{\text{L}+} = 0.999, ~~
    t_{1}^{\text{U}+} = 1.001


    t_{2}^{\text{L}-} = 1.0, ~~
    t_{2}^{\text{U}-} = 1.0, ~~
    t_{2}^{\text{L}+} = 0.998, ~~
    t_{2}^{\text{U}+} = 1.002


\begin{bmatrix}t_1^- \\ t_2^-\

# example 4

In [12]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

A₁ = randamizedIntervalPCM(A, 124, 3.5e-1)
A₂ = randamizedIntervalPCM(A, 4, 3.5e-1)

method3(A₁, A₂)

\varepsilon_1^* = 0.0

\varepsilon_2^* = 0.069

W_1^- = \begin{bmatrix} \left[ 0.512, 0.512 \right]  \\  \left[ 0.307, 0.307 \right]  \\  \left[ 0.139, 0.139 \right]  \\  \left[ 0.042, 0.042 \right] \end{bmatrix}

W_1^+ = \begin{bmatrix} \left[ 0.506, 0.512 \right]  \\  \left[ 0.211, 0.342 \right]  \\  \left[ 0.112, 0.217 \right]  \\  \left[ 0.039, 0.061 \right] \end{bmatrix}

W_2^- = \emptyset

W_2^+ = \begin{bmatrix} \left[ 0.522, 0.529 \right]  \\  \left[ 0.191, 0.352 \right]  \\  \left[ 0.092, 0.221 \right]  \\  \left[ 0.034, 0.06 \right] \end{bmatrix}

    t_{1}^{\text{L}-} = 1.0, ~~
    t_{1}^{\text{U}-} = 1.0, ~~
    t_{1}^{\text{L}+} = 1.0, ~~
    t_{1}^{\text{U}+} = 1.0


    t_{2}^{\text{L}+} = 1.0, ~~
    t_{2}^{\text{U}+} = 1.0


\begin{bmatrix}t_1^- \\ t_2^-\end{bmatrix} = \begin{bmatrix} 1.0  \\  0.0 \end{bmatrix}, ~~
\begin{bmatrix}t_1^+ \\ t_2^+\end{bmatrix} = \begin{bmatrix} 1.0  \\  1.0 \end{bmatrix}


W = \begin{bmatrix}\left[ 0.512, 0.512 \right] \\ \left[ 0.307, 0.